# Set up

The following package will download the TSV files from IMDB website and store the data into a SQLite database (requires 10GB of disk space)


In [ ]:
!pip install imdb-sqlite

In [ ]:
%%time
!imdb-sqlite

In [ ]:
# Need this package for hadling country ISO codes
!pip install pycountry

In [ ]:
# Imports

import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from matplotlib import cm # Color maps
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts

In [ ]:
# Some tweaks

warnings.filterwarnings('ignore') # Don't show warnings
sns.set_theme(style = 'whitegrid')

# Database structure analysis

In [ ]:
# Connect to databse
conn = sqlite3.connect('imdb.db')

In [ ]:
# Retrieve tables' names
tables = pd.read_sql_query('SELECT name AS "Table_name" FROM sqlite_master WHERE type="table"', conn)
tables.head()

In [ ]:
# Retrieve each table's schema

tables_names = tables['Table_name'].values.tolist()

for table_name in tables_names:
    schema = pd.read_sql_query(f'PRAGMA TABLE_INFO({table_name})', conn)
    print(f'Schema for table {table_name}:')
    display(schema)
    print(f'{50 * "_"}\n\n')

# 1. The most common content categories in IMDB

In [ ]:
# Retrieve the data
query = 'SELECT type, COUNT(*) AS abs_frequency FROM titles GROUP BY type ORDER BY abs_frequency DESC'
dataframe = pd.read_sql_query(query, conn)
display(dataframe)

In [ ]:
# Remove tvEpisode type
dataframe = dataframe.drop(labels = 0, axis = 0)

In [ ]:
# Add the relative_frequency
sum_abs_freq = dataframe['abs_frequency'].sum()
dataframe['rel_frequency'] = (dataframe['abs_frequency'] / sum_abs_freq) * 100
display(dataframe)

Now let's create a graph with 5 categories:

4 categories with the the biggest amount of titles.

1 category with the others


In [ ]:
# Others category
others = {}
others['type'] = 'others'
others['abs_frequency'] = dataframe.iloc[4:]['abs_frequency'].sum()
others['rel_frequency'] = dataframe.iloc[4:]['rel_frequency'].sum()
others

In [ ]:
# The common title categories dataframe
dataframe = dataframe.head(4)
dataframe = dataframe.append(others, ignore_index = True) # Append others category
dataframe = dataframe.sort_values(by = 'rel_frequency', ascending = False)
dataframe

In [ ]:
# Labels
labels = []

for i in dataframe.index:
    cont_type = dataframe.loc[i]["type"]
    cont_type = re.sub(r"(\w)([A-Z])", r"\1 \2", cont_type) # Add space before capital letters
    cont_type = cont_type[0].upper() + cont_type[1:] # Uppercase the first letter without changing the rest
    
    rel_freq = round(dataframe.loc[i]["rel_frequency"], 2)
    
    labels.append(f'{cont_type} {rel_freq}%')
    
labels

In [ ]:
# Plot

# Set color map
# https://matplotlib.org/stable/tutorials/colors/colormaps.html
cs = cm.tab10(range(100))

plt.pie(dataframe['rel_frequency'], colors = cs, labeldistance = 1, radius = 3, wedgeprops = dict(width = 0.8))
plt.legend(labels = labels, loc = 'center', fontsize = 12)
plt.title(label = 'Content Categories', loc = 'center', fontdict = {'fontsize':20,'fontweight':20})
plt.show()



# 2. Number of movies per genre

In [ ]:
# Retrieve movie genres and sort them
query = 'SELECT genres FROM titles WHERE type="movie"'
dataframe = pd.read_sql_query(query, conn)
display(dataframe)

In [ ]:
# Lowercase genres
dataframe['genres'] = dataframe['genres'].str.lower()

In [ ]:
# Remove missing values (\N)
dataframe['genres'].replace('\\n', np.nan, inplace = True)
genres_df = dataframe['genres'].dropna()
display(genres_df)

#### How CountVectorizer works:

![How CountVectorizer works](https://mlwhiz.com/images/countvectorizer.png)

In [ ]:
# Match words with lenght > 1 and with - or not
# We use count vectorizer to count the genderr' frequencies
pattern = '(?u)\\b\\w[\\w-]+\\b'
vect = CountVectorizer(token_pattern = pattern).fit(genres_df)
vect

In [ ]:
# Create Document-term matrix.

bag_genres = vect.transform(genres_df)
bag_genres

In [ ]:
# Return token names

unique_genres = vect.get_feature_names()
unique_genres

In [ ]:
# Sum sparse matrix columns
genres_abs_freq = bag_genres.toarray().sum(axis = 0)
genres_abs_freq

In [ ]:
# Sum of absolute frequencies
freq_total_sum = genres_abs_freq.sum()
freq_total_sum

In [ ]:
# Create final dataframe
genres_df = pd.DataFrame(zip(unique_genres, genres_abs_freq), columns = ['genre', 'abs_freq'])

In [ ]:
# Sort by absolute frequency
genres_df = genres_df.sort_values(by = 'abs_freq', ascending = False)
genres_df

In [ ]:
# Add relative frequencies to dataframe
genres_df['rel_freq'] = [(abs_freq / freq_total_sum) * 100 for abs_freq in genres_df['abs_freq']]
genres_df

In [ ]:
# Plot

plt.figure(figsize = (16, 8))
sns.barplot(x = genres_df['rel_freq'].tolist(), y = genres_df['genre'].tolist(), orient = 'h', palette = 'terrain')
plt.ylabel('Gender')
plt.xlabel('Percentual of movies (%)')
plt.title('Percentage of movie titles')
plt.show()
